In [1]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import numpy as np 
import pandas as pd
from sklearn.metrics import precision_score, recall_score

//anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
path='./'
train = pd.read_csv(path + 'input/pfm_train.csv')
test = pd.read_csv(path + 'input/pfm_test.csv')
print(train.info())
print('*' * 30)
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 31 columns):
Age                         1100 non-null int64
Attrition                   1100 non-null int64
BusinessTravel              1100 non-null object
Department                  1100 non-null object
DistanceFromHome            1100 non-null int64
Education                   1100 non-null int64
EducationField              1100 non-null object
EmployeeNumber              1100 non-null int64
EnvironmentSatisfaction     1100 non-null int64
Gender                      1100 non-null object
JobInvolvement              1100 non-null int64
JobLevel                    1100 non-null int64
JobRole                     1100 non-null object
JobSatisfaction             1100 non-null int64
MaritalStatus               1100 non-null object
MonthlyIncome               1100 non-null int64
NumCompaniesWorked          1100 non-null int64
Over18                      1100 non-null object
OverTime              

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
train.describe()

,Age,Attrition,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.0,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000
mean,36.999091,0.161818,9.427273,2.922727,1028.157273,2.725455,2.730909,2.054545,2.732727,6483.620909,2.683636,15.235455,3.152727,2.696364,80.0,0.788182,11.221818,2.807273,2.746364,7.011818,4.207273,2.226364,4.123636
std,9.037230,0.368451,8.196694,1.022242,598.915204,1.098053,0.706366,1.107805,1.109731,4715.293419,2.510017,3.628571,0.359888,1.095356,0.0,0.843347,7.825548,1.291514,0.701121,6.223093,3.618115,3.313830,3.597996
min,18.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1009.000000,0.000000,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,0.000000,2.000000,2.000000,504.250000,2.000000,2.000000,1.000000,2.000000,2924.500000,1.000000,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,0.000000,7.000000,3.000000,1026.500000,3.000000,3.000000,2.000000,3.000000,4857.000000,2.000000,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,0.000000,15.000000,4.000000,1556.500000,4.000000,3.000000,3.000000,4.000000,8354.500000,4.000000,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1.000000,29.000000,5.000000,2065.000000,4.000000,4.000000,5.000000,4.000000,19999.000000,9.000000,25.000000,4.000000,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,37.000000,18.000000,15.000000,17.000000


In [5]:
test.describe()

,Age,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.0,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000
mean,36.471429,8.391429,2.868571,1023.285714,2.714286,2.734286,2.068571,2.725714,6479.491429,2.691429,15.200000,3.160000,2.745714,80.0,0.817143,11.202857,2.782857,2.808571,6.782857,4.260000,1.951429,4.017143
std,9.373378,7.685318,1.029583,612.566819,1.067129,0.726669,1.089615,1.083437,4633.609813,2.456892,3.794884,0.367131,1.041226,0.0,0.886539,7.470399,1.295238,0.722488,5.489113,3.622336,2.752532,3.383720
min,18.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1051.000000,0.000000,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,2.000000,2.000000,463.250000,2.000000,2.000000,1.000000,2.000000,2888.750000,1.000000,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,35.000000,7.000000,3.000000,1011.000000,3.000000,3.000000,2.000000,3.000000,5104.000000,2.000000,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,42.000000,11.000000,4.000000,1584.500000,4.000000,3.000000,3.000000,4.000000,8260.250000,4.000000,18.000000,3.000000,4.000000,80.0,1.000000,15.750000,3.000000,3.000000,9.750000,7.000000,2.000000,7.000000
max,60.000000,29.000000,5.000000,2068.000000,4.000000,4.000000,5.000000,4.000000,19973.000000,9.000000,25.000000,4.000000,4.000000,80.0,3.000000,37.000000,6.000000,4.000000,29.000000,16.000000,15.000000,14.000000


In [6]:
train.corr()

,Age,Attrition,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Age,1.000000,-0.175393,0.007081,0.198558,-0.010953,0.011803,0.066528,0.513882,-0.003744,0.500163,0.291211,-0.011259,-0.029613,0.063489,NaN,-0.002413,0.682879,-0.051702,-0.001042,0.328651,0.231842,0.230587,0.212540
Attrition,-0.175393,1.000000,0.088563,-0.046494,-0.045168,-0.097003,-0.122722,-0.168775,-0.125568,-0.155521,0.025889,0.026604,0.046762,-0.051749,NaN,-0.138498,-0.187922,-0.043395,-0.048794,-0.143697,-0.163059,-0.071760,-0.158558
DistanceFromHome,0.007081,0.088563,1.000000,0.011437,0.029930,-0.010308,0.012333,0.016470,-0.009641,-0.017757,-0.016378,0.042627,0.021042,0.018112,NaN,0.050356,0.001287,-0.041208,-0.050950,0.000044,0.019317,-0.002760,0.008852
Education,0.198558,-0.046494,0.011437,1.000000,0.055979,-0.032698,0.022843,0.084075,-0.010201,0.079834,0.118484,-0.008828,-0.000045,-0.006346,NaN,0.035881,0.125672,-0.021629,0.003099,0.074522,0.064363,0.067754,0.071870
EmployeeNumber,-0.010953,-0.045168,0.029930,0.055979,1.000000,0.030428,-0.008589,-0.006121,-0.042443,-0.007147,-0.014460,0.003462,-0.023701,-0.073687,NaN,0.049967,-0.009526,0.017175,0.000548,-0.011106,-0.005649,-0.019621,-0.005333
EnvironmentSatisfaction,0.011803,-0.097003,-0.010308,-0.032698,0.030428,1.000000,-0.028467,-0.015355,0.000212,-0.026410,-0.010743,-0.008882,-0.025044,0.033515,NaN,0.008874,-0.018532,-0.045686,0.026477,-0.012574,0.003572,0.008843,-0.020190
JobInvolvement,0.066528,-0.122722,0.012333,0.022843,-0.008589,-0.028467,1.000000,0.005983,-0.016382,0.006114,0.053557,0.002377,0.000742,0.048363,NaN,0.029483,0.018380,-0.018001,-0.025862,-0.032189,0.001194,-0.031097,0.014176
JobLevel,0.513882,-0.168775,0.016470,0.084075,-0.006121,-0.015355,0.005983,1.000000,-0.005894,0.950776,0.157068,-0.066353,-0.046019,0.042156,NaN,0.002638,0.784020,-0.034620,0.041258,0.544091,0.411481,0.395195,0.376119
JobSatisfaction,-0.003744,-0.125568,-0.009641,-0.010201,-0.042443,0.000212,-0.016382,-0.005894,1.000000,-0.009752,-0.061091,0.019032,-0.011615,-0.033138,NaN,0.021123,-0.023343,0.002754,-0.042767,-0.013772,-0.011798,-0.009761,-0.041852
MonthlyIncome,0.500163,-0.155521,-0.017757,0.079834,-0.007147,-0.026410,0.006114,0.950776,-0.009752,1.000000,0.164627,-0.056469,-0.036086,0.050121,NaN,-0.006320,0.772043,-0.039713,0.032247,0.520641,0.388163,0.386875,0.344237


In [7]:
test['Attrition'] = -1
print(len(test.columns))

31


In [8]:
data = train.append(test).reset_index(drop=True)
data.head()

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,Travel_Rarely,Research & Development,1,4,Life Sciences,77,1,Male,2,2,Manufacturing Director,3,Divorced,5993,1,Y,No,18,3,3,80,1,7,2,4,7,5,0,7
1,54,0,Travel_Frequently,Research & Development,1,4,Life Sciences,1245,4,Female,3,3,Manufacturing Director,3,Divorced,10502,7,Y,No,17,3,1,80,1,33,2,1,5,4,1,4
2,34,1,Travel_Frequently,Research & Development,7,3,Life Sciences,147,1,Male,1,2,Laboratory Technician,3,Single,6074,1,Y,Yes,24,4,4,80,0,9,3,3,9,7,0,6
3,39,0,Travel_Rarely,Research & Development,1,1,Life Sciences,1026,4,Female,2,4,Manufacturing Director,4,Married,12742,1,Y,No,16,3,3,80,1,21,3,3,21,6,11,8
4,28,1,Travel_Frequently,Research & Development,1,3,Medical,1111,1,Male,2,1,Laboratory Technician,2,Divorced,2596,1,Y,No,15,3,1,80,2,1,2,3,1,0,0,0


In [9]:
path='./'
import os
import json
import gc
from tqdm import tqdm_notebook
from tqdm import tqdm
import lightgbm as lgb
import catboost as cbt
import xgboost as xgb
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score

#about time
import time
import datetime 
from datetime import datetime, timedelta
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist

from collections import Counter 
from statistics import mode 

import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast

In [10]:
#提取出类别特征的列名
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['Attrition']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    #data['count_' + i] = data.groupby([i])[i].transform('count')
    data[i] = lbl.fit_transform(data[i].astype(str))

In [11]:
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,2,1,1,4,1,77,1,1,2,2,4,3,0,5993,1,0,0,18,3,3,80,1,7,2,4,7,5,0,7
1,54,0,1,1,1,4,1,1245,4,0,3,3,4,3,0,10502,7,0,0,17,3,1,80,1,33,2,1,5,4,1,4
2,34,1,1,1,7,3,1,147,1,1,1,2,2,3,2,6074,1,0,1,24,4,4,80,0,9,3,3,9,7,0,6
3,39,0,2,1,1,1,1,1026,4,0,2,4,4,4,1,12742,1,0,0,16,3,3,80,1,21,3,3,21,6,11,8
4,28,1,1,1,1,3,3,1111,1,1,2,1,2,2,0,2596,1,0,0,15,3,1,80,2,1,2,3,1,0,0,0


In [12]:
feats = [i for i in data.columns if i not in ['Attrition']]
feats

['Age',
 'BusinessTravel',
 'Department',
 'DistanceFromHome',
 'Education',
 'EducationField',
 'EmployeeNumber',
 'EnvironmentSatisfaction',
 'Gender',
 'JobInvolvement',
 'JobLevel',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'MonthlyIncome',
 'NumCompaniesWorked',
 'Over18',
 'OverTime',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StandardHours',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

In [14]:
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=70, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary', metric= 'F1',
    subsample=0.95, colsample_bytree=0.95, subsample_freq=1,
    learning_rate=0.002, random_state=2017
    )
train_x = data[data['Attrition'] != -1][feats]
train_y = data[data['Attrition'] != -1]['Attrition']
test_x = data[data['Attrition'] == -1][feats]
# train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
model.fit(train_x, train_y)
test_pre = model.predict_proba(test_x)[:, 1] 
#num_iteration=model.best_iteration

In [15]:
%%time
#5折交叉验证
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
data.loc[data['Attrition']==2, 'Attrition'] = 0
train_x = data[data['Attrition'] != -1][feats]
train_y = data[data['Attrition'] != -1]['Attrition']
res = data[data['Attrition'] == -1][['Attrition']]
test_x = data[data['Attrition'] == -1][feats]
res['pred'] = 0

for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, train_y1, eval_set=[(train_x1, train_y1), (test_x1, test_y1)], 
              eval_metric='auc', early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:, 1]

res['pred'] = res['pred'] / 5

[1]	training's auc: 0.87362	valid_1's auc: 0.697016
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.895477	valid_1's auc: 0.718222
[3]	training's auc: 0.910167	valid_1's auc: 0.735556
[4]	training's auc: 0.919784	valid_1's auc: 0.739048
[5]	training's auc: 0.925356	valid_1's auc: 0.729841
[6]	training's auc: 0.925869	valid_1's auc: 0.732317
[7]	training's auc: 0.92756	valid_1's auc: 0.734413
[8]	training's auc: 0.929558	valid_1's auc: 0.73619
[9]	training's auc: 0.931888	valid_1's auc: 0.73619
[10]	training's auc: 0.93139	valid_1's auc: 0.735429
[11]	training's auc: 0.93227	valid_1's auc: 0.733841
[12]	training's auc: 0.9337	valid_1's auc: 0.738921
[13]	training's auc: 0.933619	valid_1's auc: 0.737778
[14]	training's auc: 0.934364	valid_1's auc: 0.731238
[15]	training's auc: 0.93374	valid_1's auc: 0.738095
[16]	training's auc: 0.93535	valid_1's auc: 0.734413
[17]	training's auc: 0.935884	valid_1's auc: 0.740381
[18]	training's auc: 0.935874	valid_1'

[191]	training's auc: 0.954404	valid_1's auc: 0.755556
[192]	training's auc: 0.954666	valid_1's auc: 0.755683
[193]	training's auc: 0.954817	valid_1's auc: 0.755556
[194]	training's auc: 0.954867	valid_1's auc: 0.755556
[195]	training's auc: 0.954998	valid_1's auc: 0.75619
[196]	training's auc: 0.955068	valid_1's auc: 0.756063
[197]	training's auc: 0.955089	valid_1's auc: 0.755683
[198]	training's auc: 0.955109	valid_1's auc: 0.755683
[199]	training's auc: 0.95526	valid_1's auc: 0.755429
[200]	training's auc: 0.95536	valid_1's auc: 0.755937
[201]	training's auc: 0.955411	valid_1's auc: 0.756063
[202]	training's auc: 0.955552	valid_1's auc: 0.755683
[203]	training's auc: 0.955602	valid_1's auc: 0.75619
[204]	training's auc: 0.955572	valid_1's auc: 0.756444
[205]	training's auc: 0.955703	valid_1's auc: 0.75619
[206]	training's auc: 0.955874	valid_1's auc: 0.75619
[207]	training's auc: 0.955954	valid_1's auc: 0.756571
[208]	training's auc: 0.955894	valid_1's auc: 0.757079
[209]	training's

[97]	training's auc: 0.935155	valid_1's auc: 0.751053
[98]	training's auc: 0.935211	valid_1's auc: 0.750877
[99]	training's auc: 0.935349	valid_1's auc: 0.752456
[100]	training's auc: 0.935506	valid_1's auc: 0.752632
[101]	training's auc: 0.935866	valid_1's auc: 0.752632
[102]	training's auc: 0.935765	valid_1's auc: 0.752807
[103]	training's auc: 0.935885	valid_1's auc: 0.751754
[104]	training's auc: 0.935755	valid_1's auc: 0.750351
[105]	training's auc: 0.935839	valid_1's auc: 0.750351
[106]	training's auc: 0.936069	valid_1's auc: 0.750351
[107]	training's auc: 0.936226	valid_1's auc: 0.750526
[108]	training's auc: 0.936355	valid_1's auc: 0.750702
[109]	training's auc: 0.936485	valid_1's auc: 0.75
[110]	training's auc: 0.936549	valid_1's auc: 0.749649
[111]	training's auc: 0.936549	valid_1's auc: 0.749298
[112]	training's auc: 0.936706	valid_1's auc: 0.749298
[113]	training's auc: 0.936798	valid_1's auc: 0.75
[114]	training's auc: 0.937112	valid_1's auc: 0.749298
[115]	training's auc:

In [16]:
pd.DataFrame({'result': res['pred']}).to_csv(path + 'output/lgb_proba.csv', index=False)

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
testX = test_x.values
trainY = train_y.values
trainX = train_x.values
model.fit(trainX, trainY)
predictY = model.predict_proba(testX)[:, 1]

In [18]:
pd.DataFrame({'result': predictY}).to_csv(path + 'output/gbdt_proba.csv',index=False)

In [19]:
#将概率值改为类别属性（0或1）并保存
res.loc[res['pred']>=0.5, 'pred'] = 1
res.loc[res['pred']<0.5, 'pred'] = 0
res.columns = ['Attrition', 'result']
res['result'] = res['result'].astype(int)
res[['result']].to_csv(path + 'output/lgb_submission_5_fold.csv', index=False)